# Сеточный мир

## Постановка задачи

Условия: задана матрица состояний 5х5, функция ценности перехода из одного состояния в другое, $\gamma = 0.9$. Стратегия выбора перехода (действия) - равновероятная.

Задача: для каждого состояния вычислить его ценность. Вычисления провести двумя методами: путём решения СЛАУ и методом Монте-Карло.

<img src="mesh_world_problem.png" width=600 height=600/>

## Решение

### Монте-Карло

Определим функцию, которая делает броски по Монте-Карло и для перехода из одного состояния в другое пользуется предоставленной стратегией.

Стратегия - это функция, которая принимает на вход текущее состояние и возвращает следующее состояние и награду за переход в следующее состояние.

In [69]:
from typing import Callable, Tuple

import numpy as np

def mc_solution(
        strategy: Callable[[int, int], Tuple[int, int, float]],
        width: int=5, 
        height: int=5, 
        gamma: float=0.9, 
        exp_cnt: int=10**6
    ):
    '''
    Monte-Carlo based solution. Model `exp_cnt` experiments, extract average out of results.
    Arguments:
    * width, height - size of statuses' matrix.
    * gamma - decay coefficient.
    * exp_cnt - number of experiments per state.
    * strategy - callable object, which accepts current state (starting from (1,1)) and returns next state (also starting from (1,1))
and reward for passing to this state.
    
    Returns numpy.ndarray of size height*width, which elements are value function of corresponding state.
    '''
    def _make_throw(i, j, gamma, gamma_i = -1):
        eps = 1/10**10
        result_i = 0.
        gamma_i = gamma if gamma_i == -1 else gamma_i
        
        next_i, next_j, reward = strategy(i, j)
        result_i += reward
        if gamma_i >= eps:
            result_i += gamma_i * _make_throw(next_i, next_j, gamma, gamma_i*gamma)
        
        return result_i
    
    result = np.zeros((height, width))
    
    for i in range(height):
        for j in range(width):
            total_value = 0.
            
            for _ in range(exp_cnt):
                total_value += _make_throw(i+1, j+1, gamma)
            result[i,j] = total_value / exp_cnt
            
    return result

Определим стратегию, которая соответствует нашей задаче.

In [61]:
import numpy as np

def strategy(i: int, j: int):
    possible_states = [
        (i+1, j),
        (i-1, j),
        (i, j+1),
        (i, j-1)
    ]
    reward = 0.
    
    next_state = possible_states[np.random.randint(0, len(possible_states))]
    
    # Check for special states.
    if i == 1 and j == 2:
        next_state = (5,2)
        reward = 10.
    elif i == 1 and j == 4:
        next_state = (3,4)
        reward = 5.
        
    # Check if we are out of grid.
    if next_state[0] < 1 or next_state[0] > 5 or \
       next_state[1] < 1 or next_state[1] > 5:
        next_state = (i,j)
        reward = -1.
    
    return *next_state, reward

In [70]:
print(mc_solution(strategy, exp_cnt=100))

[[ 2.59008863  9.29338037  4.3727448   5.15254297  0.92513858]
 [ 0.5995925   2.69892745  1.21601399  1.21520269  0.08390912]
 [-0.28401662  0.8429217   0.55434667  0.12485087 -0.53017915]
 [-0.76850065 -0.3020796  -0.31179786 -0.30295974 -1.00429402]
 [-1.35037959 -0.87185098 -0.90029531 -1.02843573 -1.36728573]]


In [71]:
print(mc_solution(strategy, exp_cnt=1000))

[[ 2.73983627  9.33731714  4.19677705  5.10553856  0.86750267]
 [ 0.70182923  2.73347021  1.66447612  1.43920198 -0.03295197]
 [-0.30296316  0.3774904   0.34254814  0.11977708 -0.50073423]
 [-0.8325209  -0.26379081 -0.2151442  -0.33771965 -0.81545179]
 [-1.32791454 -0.86352696 -0.77811689 -0.90545633 -1.38192767]]


In [ ]:
print(mc_solution(strategy, exp_cnt=10000))